In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [14]:
data = pd.read_csv('insurance_data.csv')
data.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [15]:
data['age'] = data['age']/100
data.head()

,age,affordibility,bought_insurance
0,0.22,1,0
1,0.25,0,0
2,0.47,1,1
3,0.52,0,0
4,0.46,1,1


In [16]:
X = data.drop(columns='bought_insurance')
y = data['bought_insurance']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [27]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(2,)))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros'))

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train, epochs=720)

In [33]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.6667 - loss: 0.6051


[0.6050798892974854, 0.6666666865348816]

In [35]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.8954766],
        [0.6990682]], dtype=float32),
 array([-0.5226614], dtype=float32))

In [65]:

def sigmoid(x):
    import math
    return 1/(1+np.exp(-x))

In [57]:
sigmoid(np.array([1,100,40]))

array([0.73105858, 1.        , 1.        ])

In [63]:
def prediction(age, affordability):
    w1 = 5.060867
    w2 = 1.4086502
    b = -2.9137027
    eq = w1 * age + w2 * affordability + b
    return sigmoid(eq)

In [65]:
prediction(X_test['age'], X_test['affordibility'])

2     0.705485
10    0.355695
21    0.168279
11    0.478012
14    0.726070
9     0.829498
dtype: float64

In [25]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(x,epsilon) for x in y_predicted]
    y_predicted_new = [min(x,1-epsilon) for x in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    loss = -np.mean(y_true * np.log(y_predicted_new) + (1-y_true) * np.log(1-y_predicted_new))
    return loss

In [45]:
class ANN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, age, affordability, y_true, epochs):
        self.w1, self.w2, self.bias = self.gradient_descent(age, affordability, y_true, epochs)
        return self.w1, self.w2, self.bias
        
    def predict(self, X_test):
        y_predicted = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid(y_predicted)

    def gradient_descent(self, age, affordability, y_true, epochs):
        w1 = w2 = 1
        bias = 0
        learning_rate = 0.5
        for i in range(epochs):
            y_predicted = sigmoid(w1 * age + w2 * affordability + bias)
            loss = log_loss(y_true,y_predicted)
            dw1 = np.mean(age * (y_predicted-y_true))
            dw2 = np.mean(affordability * (y_predicted-y_true))
            db = np.mean(y_predicted-y_true)
            w1 = w1 - learning_rate * dw1
            w2 = w2 - learning_rate * dw2
            bias = bias - learning_rate * db
    
        return w1, w2, bias

In [47]:
classifier = ANN()
classifier.fit(X_train['age'], X_train['affordibility'], y_train, 366)

(5.042801249636176, 1.4567027197201048, -2.956045609784402)

In [49]:
classifier.predict(X_test)

2     0.704907
10    0.356259
21    0.161797
11    0.478174
14    0.725444
9     0.828746
dtype: float64

In [23]:
X_train.iloc[20]

age              0.23
affordibility    1.00
Name: 26, dtype: float64

In [99]:
import random
from sklearn.metrics import log_loss
def stochastic_gradient_descent(X, y_true, epochs):
    n_features = X.shape[1]
    w = np.ones(n_features)
    bias = 0
    shape = len(X)
    learning_rate = 0.01
    y_true = np.asarray(y_true, dtype=float )
    for i in range(epochs):
        indices = np.arange(shape)
        np.random.shuffle(indices)
        for one_row in indices:
            # Get the row for X and y_true
            x = X.iloc[one_row]
            y = y_true[one_row]

            # Calculate the weighted sum
            weighted_sum = np.dot(w,x) + bias

            # Apply sigmoid function
            y_predicted = sigmoid(weighted_sum)

            # Calculate the log loss
            #loss = log_loss(y,y_predicted)

            # Calculate derivative for weights & bias
            dw = x * (y_predicted-y)
            db = (y_predicted-y)
            
            # Update weights & biases
            w = w - dw * learning_rate
            bias = bias - db * learning_rate

    return w, bias

In [103]:
stochastic_gradient_descent(X_train, y_train, epochs=400)

(age              3.155559
 affordibility    1.388038
 dtype: float64,
 -2.1286493010234993)

In [93]:
y_train.shape

(22,)

In [95]:
X_train.shape

(22, 2)

In [107]:
np.arange(X_train.shape[0],step = 5)

array([ 0,  5, 10, 15, 20])

In [128]:
import random
from sklearn.metrics import log_loss
def mini_batch_gradient_descent(X, y_true, epochs, batch_size):
    n_features = X.shape[1]
    w = np.ones(n_features)
    bias = 0
    shape = len(X)
    learning_rate = 0.01
    X = X.to_numpy(dtype=float) if hasattr(X, "to_numpy") else np.asarray(X, dtype=float)
    y_true = np.asarray(y_true, dtype=float )
    for i in range(epochs):
        indices = np.arange(shape)
        np.random.shuffle(indices)
        X_shuffled = X[indices]
        y_shuffled = y_true[indices]
        start = 0
        for indx in range(0,shape,batch_size):
            # Get the row for X and y_true
            x = X_shuffled[indx:indx+batch_size]
            y = y_shuffled[indx:indx+batch_size]

            m = len(y)
            if m == 0:
                continue

            # Calculate the weighted sum
            weighted_sum = np.dot(w,x.T) + bias

            # Apply sigmoid function
            y_predicted = sigmoid(weighted_sum)

            # Calculate the log loss
            #loss = log_loss(y,y_predicted)

            # Calculate derivative for weights & bias
            dw = (1/m)*np.dot(x.T, (y_predicted-y))
            db = (1/m)* np.sum(y_predicted-y)
            
            # Update weights & biases
            w = w - dw * learning_rate
            bias = bias - db * learning_rate

    return w, bias

In [121]:
stochastic_gradient_descent(X_train, y_train, epochs=400)

(age              3.155790
 affordibility    1.388212
 dtype: float64,
 -2.127064314371111)

In [130]:
mini_batch_gradient_descent(X_train, y_train, epochs=400, batch_size=5)

(array([1.34917677, 1.04208667]), -1.1052479854149537)